In [1]:
model_id = "Qwen/Qwen3-1.7B-GGUF/"
filename = "Qwen3-1.7B-Q8_0.gguf"
path = "/home/roberto/Applications/models/"+model_id
model_type = "qwen3"

In [2]:
from transformers import AutoConfig
#config = AutoConfig.from_pretrained(model_id, gguf_file=filename)
config = AutoConfig.from_pretrained(path, gguf_file=filename)
print(config)

/home/roberto/.virtualenvs/transformers/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


LlamaConfig {
  "_model_name_or_path": "LLaMA v2",
  "attention_bias": false,
  "attention_dropout": 0.0,
  "bos_token_id": 1,
  "eos_token_id": 2,
  "head_dim": 128,
  "hidden_act": "silu",
  "hidden_size": 4096,
  "initializer_range": 0.02,
  "intermediate_size": 11008,
  "max_position_embeddings": 4096,
  "mlp_bias": false,
  "model_type": "llama",
  "num_attention_heads": 32,
  "num_hidden_layers": 32,
  "num_key_value_heads": 32,
  "pretraining_tp": 1,
  "rms_norm_eps": 9.999999747378752e-06,
  "rope_scaling": null,
  "rope_theta": 10000.0,
  "tie_word_embeddings": false,
  "transformers_version": "4.55.3",
  "unk_token_id": 0,
  "use_cache": true,
  "vocab_size": 32000
}



In [3]:
from ctransformers import AutoModelForCausalLM
model = AutoModelForCausalLM.from_pretrained(path, model_file=filename, model_type=model_type, hf=True)
#model = AutoModelForCausalLM.from_pretrained(model_id, gguf_file=filename)
type(model)

CTransformersModel has generative capabilities, as `prepare_inputs_for_generation` is explicitly defined. However, it doesn't directly inherit from `GenerationMixin`. From 👉v4.50👈 onwards, `PreTrainedModel` will NOT inherit from `GenerationMixin`, and this model will lose the ability to call `generate` and other related functions.
  - If you're using `trust_remote_code=True`, you can get rid of this warning by loading the model with an auto class. See https://huggingface.co/docs/transformers/en/model_doc/auto#auto-classes
  - If you are the owner of the model architecture code, please modify your model class such that it inherits from `GenerationMixin` (after `PreTrainedModel`, otherwise you'll get an exception).
  - If you are not the owner of the model architecture class, please contact the model code owner to update it.


ctransformers.transformers.CTransformersModel

In [4]:
from transformers import AutoTokenizer
#tokenizer = AutoTokenizer.from_pretrained(model_id, gguf_file=filename)
tokenizer = AutoTokenizer.from_pretrained(path, gguf_file=filename)
encoding=tokenizer("I love you")
print(encoding)

You are using the default legacy behaviour of the <class 'transformers.models.llama.tokenization_llama_fast.LlamaTokenizerFast'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565 - if you loaded a llama tokenizer from a GGUF file you can ignore this message.
Merges were not in checkpoint, building merges on the fly.
100%|████████████████████████████████████| 32000/32000 [01:22<00:00, 389.38it/s]


{'input_ids': [1, 306, 5360, 366], 'attention_mask': [1, 1, 1, 1]}


In [5]:
from transformers import pipeline
pipeline = pipeline(task="text-generation", model=model, tokenizer=tokenizer, device_map="auto")
chat = [
    {"role": "system", "content": "You are a helpful science assistant."},
    {"role": "user", "content": "What's the name of an Irish music band that has an album titled: How to dismantle an atomic bomb?"}
]
prompt = pipeline.tokenizer.apply_chat_template(chat, tokenize=False, add_generation_prompt=True)
print(prompt)

Device set to use cpu
CTransformersModel has generative capabilities, as `prepare_inputs_for_generation` is explicitly defined. However, it doesn't directly inherit from `GenerationMixin`. From 👉v4.50👈 onwards, `PreTrainedModel` will NOT inherit from `GenerationMixin`, and this model will lose the ability to call `generate` and other related functions.
  - If you're using `trust_remote_code=True`, you can get rid of this warning by loading the model with an auto class. See https://huggingface.co/docs/transformers/en/model_doc/auto#auto-classes
  - If you are the owner of the model architecture code, please modify your model class such that it inherits from `GenerationMixin` (after `PreTrainedModel`, otherwise you'll get an exception).
  - If you are not the owner of the model architecture class, please contact the model code owner to update it.


AttributeError: 'TextGenerationPipeline' object has no attribute 'assistant_model'

In [ ]:
response = pipeline(chat, max_new_tokens=32)
print(response[0]["generated_text"])

In [6]:
del model, tokenizer

In [7]:
del pipeline

In [8]:
import gc
gc.collect()
#torch.cuda.empty_cache()

1160